In [8]:
import os
os.environ["PATH"] += os.pathsep + r'C:\Program Files\Graphviz\bin'

from diagrams import Cluster, Diagram, Edge
from diagrams.onprem.analytics import Spark
from diagrams.onprem.compute import Server
from diagrams.onprem.database import PostgreSQL
from diagrams.onprem.inmemory import Redis
from diagrams.onprem.logging import Fluentd
from diagrams.onprem.monitoring import Grafana, Prometheus
from diagrams.onprem.network import Nginx
from diagrams.onprem.queue import Kafka

with Diagram(name="Advanced Web Service with On-Premise (colored)", show=True):
    ingress = Nginx("ingress")

    metrics = Prometheus("metric")
    metrics << Edge(color="firebrick", style="dashed") << Grafana("monitoring")

    with Cluster("Service Cluster"):
        grpcsvc = [
            Server("grpc1"),
            Server("grpc2"),
            Server("grpc3")]

    with Cluster("Sessions HA"):
        master = Redis("session")
        master - Edge(color="brown", style="dashed") - Redis("replica") << Edge(label="collect") << metrics
        grpcsvc >> Edge(color="brown") >> master

    with Cluster("Database HA"):
        master = PostgreSQL("users")
        master - Edge(color="brown", style="dotted") - PostgreSQL("slave") << Edge(label="collect") << metrics
        grpcsvc >> Edge(color="black") >> master

    aggregator = Fluentd("logging")
    aggregator >> Edge(label="parse") >> Kafka("stream") >> Edge(color="black", style="bold") >> Spark("analytics")

    ingress >> Edge(color="darkgreen") << grpcsvc >> Edge(color="darkorange") >> aggregator

In [10]:
from diagrams import Cluster, Diagram
from diagrams.k8s.compute import Pod, StatefulSet
from diagrams.k8s.network import Service
from diagrams.k8s.storage import PV, PVC, StorageClass

with Diagram("Stateful Architecture", show=True):
    with Cluster("Apps"):
        svc = Service("svc")
        sts = StatefulSet("sts")

        apps = []
        for _ in range(3):
            pod = Pod("pod")
            pvc = PVC("pvc")
            pod - sts - pvc
            apps.append(svc >> pod >> pvc)

    apps << PV("pv") << StorageClass("sc")

In [11]:
from diagrams import Diagram
from diagrams.k8s.clusterconfig import HPA
from diagrams.k8s.compute import Deployment, Pod, ReplicaSet
from diagrams.k8s.network import Ingress, Service

with Diagram("Exposed Pod with 3 Replicas", show=True):
    net = Ingress("domain.com") >> Service("svc")
    net >> [Pod("pod1"),
            Pod("pod2"),
            Pod("pod3")] << ReplicaSet("rs") << Deployment("dp") << HPA("hpa")

In [13]:
from diagrams import Cluster, Diagram
from diagrams.gcp.analytics import BigQuery, Dataflow, PubSub
from diagrams.gcp.compute import AppEngine, Functions
from diagrams.gcp.database import BigTable
from diagrams.gcp.iot import IotCore
from diagrams.gcp.storage import GCS

with Diagram("Message Collecting", show=True):
    pubsub = PubSub("pubsub")

    with Cluster("Source of Data"):
        [IotCore("core1"),
         IotCore("core2"),
         IotCore("core3")] >> pubsub

    with Cluster("Targets"):
        with Cluster("Data Flow"):
            flow = Dataflow("data flow")

        with Cluster("Data Lake"):
            flow >> [BigQuery("bq"),
                     GCS("storage")]

        with Cluster("Event Driven"):
            with Cluster("Processing"):
                flow >> AppEngine("engine") >> BigTable("bigtable")

            with Cluster("Serverless"):
                flow >> Functions("func") >> AppEngine("appengine")

    pubsub >> flow

In [14]:
from diagrams import Cluster, Diagram
from diagrams.aws.compute import ECS, EKS, Lambda
from diagrams.aws.database import Redshift
from diagrams.aws.integration import SQS
from diagrams.aws.storage import S3

with Diagram("Event Processing", show=True):
    source = EKS("k8s source")

    with Cluster("Event Flows"):
        with Cluster("Event Workers"):
            workers = [ECS("worker1"),
                       ECS("worker2"),
                       ECS("worker3")]

        queue = SQS("event queue")

        with Cluster("Processing"):
            handlers = [Lambda("proc1"),
                        Lambda("proc2"),
                        Lambda("proc3")]

    store = S3("events store")
    dw = Redshift("analytics")

    source >> workers >> queue >> handlers
    handlers >> store
    handlers >> dw

In [44]:
list(df['Task'])

['Results of Task',
 'Establish and agree upon Agile standards/framework',
 'Communicate Agile structure to business groups',
 'Select proper tooling',
 'Results of Task',
 'Refine analytic data use operating model',
 'Define analytic persona\x92s and use cases',
 'Define operational data use operating model',
 'Define operational persona\x92s and use cases ',
 'Define standard ',
 'Results of Task',
 'ID appropriate use case requirements for applying methodology (i.e. what are expectations from the business, what are deliverable expectations etc.)',
 'Create template for data use stories (i.e. data sets, data presentation, data analysis, Data validation, advanced analytic stories)',
 'Results of Task',
 'Identify product advisory council attendees',
 'Create governance model/methodology',
 'Results of Task',
 'ID all candidate roles and process to keep bench as opposed to OJT',
 'ID incentive and disincentive components to manage contract',
 'ID candidate partners for various work com

In [200]:
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objs as go
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

df = pd.read_csv('gantt_chart_new.csv',encoding='latin1')
df['Start'] = df['Start'].astype('datetime64')
df['Finish'] = df['Finish'].astype('datetime64')
# print(df.dtypes)
# print(df.head())

colors = {'Technology' : 'rgb(30,144,255)'
          , 'Technology - Date TBD' : 'rgb(211,211,211)'
          , 'People' : 'rgb(95,158,160)'
          , 'Process' : 'rgb(0,0,128)'
          , 'Process - Date TBD' : 'rgb(211,211,210)'}
    
orders = list(df['Task'])

fig = px.timeline(df
                  , x_start="Start"
                  , x_end="Finish"
                  , y="Resource"
                  , hover_name="Task"
#                   , facet_col="Dimension"
#                   , facet_col_wrap=40
#                   , facet_col_spacing=.99
#                   , color_discrete_sequence=['green']*len(df)
                  , color_discrete_sequence=px.colors.qualitative.Prism
                  , category_orders={'Task':['Results of Task','Establish and agree upon Agile standards/framework','Communicate Agile structure to business groups','Select proper tooling','Results of Task','Refine analytic data use operating model','Define analytic persona\x92s and use cases','Define operational data use operating model','Define operational persona\x92s and use cases ','Define standard ','Results of Task','ID appropriate use case requirements for applying methodology (i.e. what are expectations from the business, what are deliverable expectations etc.)',
                                     'Create template for data use stories (i.e. data sets, data presentation, data analysis, Data validation, advanced analytic stories)','Results of Task','Identify product advisory council attendees','Create governance model/methodology','Results of Task','ID all candidate roles and process to keep bench as opposed to OJT','ID incentive and disincentive components to manage contract','ID candidate partners for various work components','Create methodology to manage bid process and selection \x96 partner with procurement','Results of task','Define scope of project','Results of task','ID tools for self-service ties to tools/tech','Determine extent of centrally managed policy engines','What multi-tenancy restrictions should be established','Access control \x96 Tie to IT IAM strategy','Evaluate security, PHI, access restrictions','Results of task','Publish product and service catalog outlining initial build/implementation of the product or service as well as the ongoing costs','Results of task','ID standard tools for self-service','Create governance and methodology use cases','Create training protocol for tool consumption',
                                     'Database/System design for persistent data to be stored on remote, network attached storage','Identify systems requirements','Technology/vendor selection','Migration effort to be considered; layout strategy','Downstream efforts - existing processes to migrate; how to enable a seamless transition; which approach to use - coexist or turn off/turn on','Enable S3 as primary storage, move away from other storage solutions and determine config','Establish rules/processes to ensure S3 is the primary storage solution','Recompilation and redeployment of jobs is required','Stand up non-prod two node cluster job/script execution','Decommission version 5 by May 21 2021','Results of task','Identify self-service needs/requirements for each group ','Identify the right tool for the right purpose','Learning and creating guidelines on how to use new tools','Estimate adoption rate','Gauge time to ramp-up','Evaluate if migration from existing tools to new tools is required',
                                     'Results of task','Identify compatible tooling','Determine requirements specific for each business group','Identify VDI-based security issues for additional R/Python libraries','Document and communicate procedure to add new ML features/libraries to remote environments','Estimate value of paid tools vs free libraries','Determine if automated ML tools are necessary (Datarobot vs Azure ML)','Gauge level of adoption','Document FAQs and consider questions specific to each business group','Results of task','Determine scope \x96 DR, HA, Extent of insight and notification needs','Establish continuous monitoring needs aligned with DevOps','Resiliency testing approach','Monitoring tools','Incident response communication protocol (ties to governance and operating model)','Identify compatible tooling/technology','Consider location/region-based connection practices',
                                     'Consider multiple physical data center connection locations for resiliency to connectivity failures due to fiber cut, device failure or complete location failure','Determine if dynamically routed and active connections vs. statically routed for load balancing and failover','Results of task','Identify compatible tooling','Determine instance requirements specific for each business group','Launch instances and document keys/passwords','Communicate instance configurations and setup notes to each group','Estimate level of adoption','Predict and document FAQs','Document how to use Microsoft Remote Desktop Apps for Mac users','Install supporting software (R, Python, Firefox etc.)','Identify an owner/manager of compute elasticity/infrastructure operations',
                                     'Identify rules of engagement and governance for when the process is live','Results of task','Prioritize data pipelines (tie to Data COE)','Pilot analytics in data pipeline (tie to NextBlue AA COE)','Define event streaming hub approach (i.e. people process tech)','Determine appropriate/compatible event streaming tools (Apache Kafka, Hazelcast Jet etc.)','Determine streaming rate requirements specific to each business group','Map out the data flow/data architecture and associated patterns','Identify who will do the publishing framework','Deploy and document','There are dependencies on infrastructure prior to putting this in place','Results of task','Define tools/tech and associated dependencies','Align with cloud COE','Establish development guidelines','Pilot?','Tie to governance and operating model \x96 when to use how to use etc.','Determine which tasks will be microservices','Determine tools to run/containerize microservices',
                                     'Select a microservice deployment strategy (Service Instance per Host, Multiple Service Instances per Host, Service Instance per Virtual Machine etc.)','Determine if you need a cluster manager (Kubernetes, Marathon etc.)','Identify enablement approach','Results of task','Define tools/tech and associated dependencies','Align with cloud COE','Establish development guidelines','Pilot?','Tie to governance and operating model \x96 when to use how to use etc.','Determine whether to containerize existing legacy application without refactoring into microservices','Identify barriers between security and devops','Determine drawbacks: such as container infrastructure is not as mature or secure as the infrastructure for VMs (containers share the kernel of the host OS with one another)','Results of task','Identify a back-end physical infrastructure to enable digital tools','Evaluate a hybrid cloud strategy (with an eye for the worker UX)',
                                     'Evaluate existing support services for the workplace','Implement technology for a real-time, 360 view of the UX','Results of task','Orchestration and management of services, applications and data flow','Automation of business processes','Establishment of federated security and Single Sign-On','System logging and monitoring','Data analysis, reporting and predictions','Monitoring of mobile and IoT devices','Implementation of Big Data solutions','Centralized (mobile ready) web solutions for clients and employees','Scalability and high-load support','SaaS and multi-tenancy support','Centralized control of infrastructure management etc.','Results of task','Determine compatible tools','Identify downfalls of low code platforms','Estimate adoption rate',
                                     'Results of task','Determine the FHIR server architecture','Determine compatibility of servers and FHIR versions to ensure interoperability','Understand the APIs as well as the level of completion per API','Identify where to put/install the FHIR server','Estimate adoption rate','Results of task','ID priority governance model evaluation needs \x96 self-service, machine learning model productionalization, operational area use and interaction']}
                  , opacity=.7
#                   , text="Task"
                  , range_x=None
                  , range_y=None
                  , template='plotly_white'
                  , height=800
                  , width=1500
                  , color='Dimension'
                  , title ="<b>IE 3.0 Gantt Chart 2021</b>"
#                   , color=colors
                 )

fig.update_layout(
    bargap=0.5
    ,bargroupgap=0.1
    ,xaxis_range=[df.Start.min(), df.Finish.max()]
    ,xaxis = dict(
        showgrid=True
        ,rangeslider_visible=True
        ,side ="top"
        ,tickmode = 'array'
        ,dtick="M1"
        ,tickformat="Q%q %Y \n"
        ,ticklabelmode="period"        
        ,ticks="outside"
        ,tickson="boundaries"
        ,tickwidth=.1
        ,layer='below traces'
        ,ticklen=20
        ,tickfont=dict(
            family='Old Standard TT, serif',size=24,color='gray')
        ,rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
            ,x=.37
            ,y=-.05
            ,font=dict(
                family="Arial",
                size=14,
                color="darkgray"
    )))
    
    ,yaxis = dict(
        title= ""
        ,autorange="reversed"
        ,automargin=True
#         ,anchor="free"
        ,ticklen=10
        ,showgrid=True
        ,showticklabels=True
        ,tickfont=dict(
            family='Old Standard TT, serif', size=16, color='gray'))
    
    ,legend=dict(
        orientation="h"
        ,yanchor="bottom"
        ,y=1.1
        ,title=""
        ,xanchor="right"
        ,x=1
        ,font=dict(
            family="Arial"
            ,size=14
            ,color="darkgray"))
)

fig.update_traces( #marker_color='rgb(158,202,225)'
                   marker_line_color='rgb(8,48,107)'
                  , marker_line_width=1.5, opacity=0.95)

fig.update_layout(
    title="<b>IE 3.0 Gantt Chart 2021</b>",
    xaxis_title="",
#     margin_l=400,
    yaxis_title="Initiatives",
#     legend_title="Dimension: ",
    font=dict(
        family="Arial",
        size=24,
        color="darkgray"
    )
)

# fig.show()
fig.write_html("C:/Users/maxwell.bade/Downloads/ie_3_gantt.html")
go.FigureWidget(fig)

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'base': array([datetime.datetime(2021, 3,…

In [202]:
username = 'maxbade'
api_key = 'ZLoHQuLuAbT6BlsApZYK'

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
py.plot(fig, filename = 'IE 3.0 Gantt Chart Plotly', auto_open=False)

'https://plotly.com/~maxbade/1/'

In [19]:
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff

df = pd.read_csv('bcbsm_gantt.csv',encoding='latin1')
df['Start'] = df['Start'].astype('datetime64')
df['Finish'] = df['Finish'].astype('datetime64')
# print(df.dtypes)
# print(df.head())

colors = {'Technology' : 'rgb(30,144,255)'
          , 'Technology - Date TBD' : 'rgb(211,211,211)'
          , 'People' : 'rgb(95,158,160)'
          , 'Process' : 'rgb(0,0,128)'
          , 'Process - Date TBD' : 'rgb(211,211,210)'}

['ggplot2', 'seaborn', 'simple_white', 'plotly',
'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
'ygridoff', 'gridon', 'none']
#         ,ticktext=["Q1", "Q2", "Q3", "Q4"]
#         ,tickformat="Q%q %Y \n %b"        
#         ,tickvals = pd.date_range('2021-01-01', '2021-12-31', freq='Q')
#         ,tickformat="Q%q %Y \n %b \n"
#         ,nticks=4
#         ,tickformat = 'End of Q%q <br> %d %B <br>%Y'

fig = px.timeline(df
                  , x_start="Start"
                  , x_end="Finish"
                  , y="Task"
#                   , facet_col="Dimension"
#                   , facet_col_wrap=40
#                   , facet_col_spacing=.99
#                   , color_discrete_sequence=['green']*len(df)
                  , color_discrete_sequence=px.colors.qualitative.Prism
                  , category_orders={"Task": ['Agile DevOps Culture','Organizational Design and Operating Model',
                                     'Delivery Methodology-Agile, RAD, DevOps, Left Testing, TDD, Low-Code',
                                     'Product Mgmt Evolution-Product Advisory Council','Expansion of Managed Service Arrangements',
                                     'DevOps, DataOps, MLOps','Expansion of Self-Service Models','Financial Transparency Process Dev','Decouple compute/storage',
                                     'Self-Service Tools','AI/ML Infrastructure/tools','Resilient/Durable',
                                     'Elastic Infrastructure/Dynamic Compute','Event Streaming','Microservices','Containerization ',
                                     'Hybrid Cloud Governance, Security and UX, Data Arch, Data Quality Mgmt',
                                     'Enterprise Orchestration','Low-Code Dev Tools','FHIR Data Architecture',
                                     'Governance Model Dev-Self-Service, Productionization, Lineage']}
                  , opacity=.5
                  , range_x=None
                  , range_y=None
                  , template='seaborn'
                  , height=1000
                  , color='Dimension'
#                   , color=colors
                 )

fig.update_layout(
#     title ="<b>IE 3.0 Gantt Chart 2021</b>"
    xaxis_range=[df.Start.min(), df.Finish.max()]
    ,xaxis = dict(
        showgrid=True
        ,rangeslider_visible=True
        ,side ="top"
        ,tickmode = 'array'
#         ,ticktext=["Q1", "Q2", "Q3", "Q4"]
        ,tickvals = pd.date_range('2021-01-01', '2021-12-31', freq='Q')
        ,tickformat = 'End of Q%q <br> %d %B <br>%Y'
        ,ticks="outside"
        ,tickson="boundaries"
        ,tickwidth=.1
#         ,ticklabelmode='period'
        ,layer='below traces'
        ,ticklen=2
        ,tickfont=dict(
            family='Old Standard TT, serif',
            size=16,
            color='darkgray'))
    ,yaxis = dict(
        title= ""
        ,automargin=True
        ,ticklen=3
        ,showgrid=True
        ,showticklabels=True
        ,tickfont=dict(
            family='Old Standard TT, serif',
            size=12,
            color='black'))
    ,legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.1,
        title="",
        xanchor="right",
        x=1)
)

#annotations
agile_result = "Adopt an Agile/DevOps Cultre"
org_design = "More Aligned to business needs"
delivery_method = "Visibility of capacity/tasks/outcomes"
prod_mgmt = "Better IE collaboration"
expansion = "Elastic cpacity to serve more customers"
devops = "Automation and greater efficiency"
expand_self_serve= "Greater flexibility and autonomy"
financial = "Address Cost Transparency"
decouple = "Greater flexibility, elasticity and tracking"
self_service = "Greater access and usage to IE"
ai_ml = "Streamlined model dev and operations"
resilient = "Avoid application failures and downtime"
elastic = "Elastic and flexible for complex workloads"
event_streaming = "Consistent, real-time, data ingestion"
microservices = "Enable distributed data integration"
containerization = "Consumption-based cost, portability, consisten environments"
hybrid = "Align cloud and networking strategies and simplify access controls implementation"
enterprise_orch = "Resolved data latency and interoperability"
low_code = "Business-friendly tools and requires a less technical skillset"
fhir = "Support bot clinical and non-clinical data"
governance = "Greater ability for self-service and enhanced data lineage"
fig['layout']['annotations'] += tuple([dict(x='2021-02-25',y=20,text=agile_result, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-02-25',y=19,text=org_design, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-02-22',y=18,text=delivery_method, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-03-08',y=17,text=prod_mgmt, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-02-19',y=16,text=expansion, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-05-25',y=15,text=devops, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-08-28',y=14,text=expand_self_serve, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-12-02',y=13,text=financial, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-11-18',y=12,text=decouple, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-11-28',y=11,text=self_service, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-08-22',y=10,text=ai_ml, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-08-20',y=9,text=resilient, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-11-19',y=8,text=elastic, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-11-22',y=7,text=event_streaming, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-11-25',y=6,text=microservices, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-10-30',y=5,text=containerization, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-10-09',y=4,text=hybrid, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-11-18',y=3,text=enterprise_orch, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-11-01',y=2,text=low_code, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-11-19',y=1,text=fhir, showarrow=False, font=dict(color='black', size=10))])
fig['layout']['annotations'] += tuple([dict(x='2021-11-01',y=0,text=governance, showarrow=False, font=dict(color='black', size=10))])

fig.show()

In [476]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id="graph"),
    dcc.Checklist(
        id='tick',
        options=[{'label': 'Enable Linear Ticks', 
                  'value': 'linear'}],
        value=['linear']
    )
])

@app.callback(
    Output("graph", "figure"), 
    [Input("tick", "value")])
def display_figure(tick_mode):
    fig = px.timeline(df
                  , x_start="Start"
                  , x_end="Finish"
                  , y="Task"
                  , color_discrete_sequence=px.colors.qualitative.Alphabet
                  , category_orders={"Task": ['Agile DevOps Culture',
                                     'Organizational Design and Operating Model',
                                     'Delivery Methodology-Agile, RAD, DevOps, Left Testing, TDD, Low-Code',
                                     'Product Mgmt Evolution-Product Advisory Council',
                                     'Expansion of Managed Service Arrangements',
                                     'DevOps, DataOps, MLOps',
                                     'Expansion of Self-Service Models',
                                     'Financial Transparency Process Dev',
                                     'Decouple compute/storage',
                                     'Self-Service Tools',
                                     'AI/ML Infrastructure/tools',
                                     'Resilient/Durable',
                                     'Elastic Infrastructure/Dynamic Compute',
                                     'Event Streaming',
                                     'Microservices',
                                     'Containerization ',
                                     'Hybrid Cloud Governance, Security and UX, Data Arch, Data Quality Mgmt',
                                     'Enterprise Orchestration',
                                     'Low-Code Dev Tools',
                                     'FHIR Data Architecture',
                                     'Governance Model Dev-Self-Service, Productionization, Lineage']}
                  , opacity=.5
                  , range_x=None
                  , range_y=None
                  , template='presentation'
                  , height=1000
                  , color='Dimension'
                 )
    
    fig.update_layout(
        xaxis_range=[df.Start.min(), df.Finish.max()]
        ,xaxis = dict(
            showgrid=True
            ,rangeslider_visible=True
            ,side ="top"
            ,tickmode = 'array'
            ,tickvals = pd.date_range('2021-01-01', '2021-12-31', freq='Q')
            ,tickformat = 'End of Q%q <br> %d %B <br>%Y'
            ,ticks="outside"
            ,tickson="boundaries"
            ,tickwidth=.1
            ,layer='below traces'
            ,ticklen=2
            ,tickfont=dict(
                family='Old Standard TT, serif',
                size=16,
                color='darkgray'))
        ,yaxis = dict(
            title= ""
            ,automargin=True
            ,ticklen=3
            ,showgrid=True
            ,showticklabels=True
            ,tickfont=dict(
                family='Old Standard TT, serif',
                size=12,
                color='black'))
        ,legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.1,
            title="",
            xanchor="right",
            x=1))

    #annotations
    agile_result = "Adopt an Agile/DevOps Cultre"
    org_design = "More Aligned to business needs"
    delivery_method = "Visibility of capacity/tasks/outcomes"
    prod_mgmt = "Better IE collaboration"
    expansion = "Elastic cpacity to serve more customers"
    devops = "Automation and greater efficiency"
    expand_self_serve= "Greater flexibility and autonomy"
    financial = "Address Cost Transparency"
    decouple = "Greater flexibility, elasticity and tracking"
    self_service = "Greater access and usage to IE"
    ai_ml = "Streamlined model dev and operations"
    resilient = "Avoid application failures and downtime"
    elastic = "Elastic and flexible for complex workloads"
    event_streaming = "Consistent, real-time, data ingestion"
    microservices = "Enable distributed data integration"
    containerization = "Consumption-based cost, portability, consisten environments"
    hybrid = "Align cloud and networking strategies and simplify access controls implementation"
    enterprise_orch = "Resolved data latency and interoperability"
    low_code = "Business-friendly tools and requires a less technical skillset"
    fhir = "Support bot clinical and non-clinical data"
    governance = "Greater ability for self-service and enhanced data lineage"
    fig['layout']['annotations'] += tuple([dict(x='2021-02-25',y=20,text=agile_result, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-02-25',y=19,text=org_design, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-02-22',y=18,text=delivery_method, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-03-08',y=17,text=prod_mgmt, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-02-19',y=16,text=expansion, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-05-25',y=15,text=devops, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-08-28',y=14,text=expand_self_serve, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-12-02',y=13,text=financial, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-11-18',y=12,text=decouple, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-11-28',y=11,text=self_service, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-08-22',y=10,text=ai_ml, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-08-20',y=9,text=resilient, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-11-19',y=8,text=elastic, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-11-22',y=7,text=event_streaming, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-11-25',y=6,text=microservices, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-10-30',y=5,text=containerization, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-10-09',y=4,text=hybrid, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-11-18',y=3,text=enterprise_orch, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-11-01',y=2,text=low_code, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-11-19',y=1,text=fhir, showarrow=False, font=dict(color='black', size=10))])
    fig['layout']['annotations'] += tuple([dict(x='2021-11-01',y=0,text=governance, showarrow=False, font=dict(color='black', size=10))])

    return fig

app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

In [185]:
df = pd.read_csv('bcbsm_gantt.csv',encoding='latin1')

In [186]:
df['Start'] = df['Start'].astype('datetime64')
df['Finish'] = df['Finish'].astype('datetime64')
print(df.dtypes)
print(df.head())

Task                 object
Start        datetime64[ns]
Finish       datetime64[ns]
Dimension            object
dtype: object
                                                Task      Start     Finish  \
0                           Decouple compute/storage 2021-01-01 2021-12-31   
1                                  Containerization  2021-01-01 2021-12-31   
2  Hybrid Cloud Governance, Security and UX, Data... 2021-01-01 2021-12-31   
3                           Enterprise Orchestration 2021-01-01 2021-12-31   
4                                 Low-Code Dev Tools 2021-01-01 2021-12-31   

               Dimension  
0             Technology  
1  Technology - Date TBD  
2  Technology - Date TBD  
3  Technology - Date TBD  
4  Technology - Date TBD  


In [8]:
# Gantt chart in Plotly has mandatory format of data columns
df.columns = ['Task', 'Start', 'Finish', 'Dimension']
# tasks_data_plotly['Resource'] = [ 'Critical' if el else 'Not critical' for el in tasks_data_plotly['Resource']]

colors = {'Technology' : 'rgb(30,144,255)'
          , 'Technology - Date TBD' : 'rgb(211,211,211)'
          , 'People' : 'rgb(95,158,160)'
          , 'Process' : 'rgb(0,0,128)'
          , 'Process - Date TBD' : 'rgb(211,211,210)'}

fig_plotly = ff.create_gantt(df
                             , colors=colors
                             , index_col='Dimension'
                             , title='Gantt Chart'
                             , show_colorbar=True
                             , bar_width=0.4
                             , showgrid_x=True
                             , showgrid_y=True
                            )
fig_plotly.show()

ValueError: Length mismatch: Expected axis has 5 elements, new values have 4 elements

In [244]:
colors = {'Technology' : 'rgb(30,144,255)'
          , 'Technology - Date TBD' : 'rgb(211,211,211)'
          , 'People' : 'rgb(95,158,160)'
          , 'Process' : 'rgb(0,0,128)'
          , 'Process - Date TBD' : 'rgb(211,211,210)'}

fig = px.timeline(df
                  , x_start="Start"
                  , x_end="Finish"
                  , y="Task"
#                   , color_discrete_sequence=['green']*len(df)
                  , color_discrete_sequence=px.colors.qualitative.Alphabet
                  , opacity=.5
                  , range_x=None
                  , range_y=None
                  , template='ggplot2'
#                   , width=1000
                  , height=700
#                   , labels={'x': 'Some X', 'y':'Some Y'}
                  , color='Dimension'
#                   , color=colors
                 )

fig.update_layout(
    title = "<b>IE 3.0 Gantt Chart 2021</b>"
    ,xaxis_range=[df.Start.min(), df.Finish.max()]
    ,xaxis = dict(
        showgrid=True
        ,rangeslider_visible=True
#         ,title='<b>Quarter</b>'
        ,tickmode = 'array'
        ,ticktext=["Q1", "Q2", "Q3", "Q4"]
        ,tickvals = pd.date_range('2021-01-01', '2021-12-31', freq='Q')
        ,tickformat = '%b'
        ,ticks="outside"
        ,tickson="labels"
        ,tickfont=dict(
            family='Old Standard TT, serif',
            size=20,
            color='black'))
    ,yaxis = dict(
        title='<b>Initiatives</b>'
        ,automargin=True
        ,showgrid=True
        ,showticklabels=True
        ,tickfont=dict(
            family='Old Standard TT, serif',
            size=12,
            color='black'))
    ,legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.0,
        xanchor="right",
        x=1)
)

fig.show()

In [ ]:
    
['ggplot2', 'seaborn', 'simple_white', 'plotly',
'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
'ygridoff', 'gridon', 'none']
    
    - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darksalmon, darkseagreen,
            darkslateblue, darkslategray, darkslategrey,
            darkturquoise, darkviolet, deeppink, deepskyblue,
            dimgray, dimgrey, dodgerblue, firebrick,
            floralwhite, forestgreen, fuchsia, gainsboro,
            ghostwhite, gold, goldenrod, gray, grey, green,
            greenyellow, honeydew, hotpink, indianred, indigo,
            ivory, khaki, lavender, lavenderblush, lawngreen,
            lemonchiffon, lightblue, lightcoral, lightcyan,
            lightgoldenrodyellow, lightgray, lightgrey,
            lightgreen, lightpink, lightsalmon, lightseagreen,
            lightskyblue, lightslategray, lightslategrey,
            lightsteelblue, lightyellow, lime, limegreen,
            linen, magenta, maroon, mediumaquamarine,
            mediumblue, mediumorchid, mediumpurple,
            mediumseagreen, mediumslateblue, mediumspringgreen,
            mediumturquoise, mediumvioletred, midnightblue,
            mintcream, mistyrose, moccasin, navajowhite, navy,
            oldlace, olive, olivedrab, orange, orangered,
            orchid, palegoldenrod, palegreen, paleturquoise,
            palevioletred, papayawhip, peachpuff, peru, pink,
            plum, powderblue, purple, red, rosybrown,
            royalblue, rebeccapurple, saddlebrown, salmon,
            sandybrown, seagreen, seashell, sienna, silver,
            skyblue, slateblue, slategray, slategrey, snow,
            springgreen, steelblue, tan, teal, thistle, tomato,
            turquoise, violet, wheat, white, whitesmoke,
            yellow, yellowgreen

In [ ]:
Valid properties:
        anchor
            If set to an opposite-letter axis id (e.g. `x2`, `y`),
            this axis is bound to the corresponding opposite-letter
            axis. If set to "free", this axis' position is
            determined by `position`.
        automargin
            Determines whether long tick labels automatically grow
            the figure margins.
        autorange
            Determines whether or not the range of this axis is
            computed in relation to the input data. See `rangemode`
            for more info. If `range` is provided, then `autorange`
            is set to False.
        calendar
            Sets the calendar system to use for `range` and `tick0`
            if this is a date axis. This does not set the calendar
            for interpreting data on this axis, that's specified in
            the trace or via the global `layout.calendar`
        categoryarray
            Sets the order in which categories on this axis appear.
            Only has an effect if `categoryorder` is set to
            "array". Used with `categoryorder`.
        categoryarraysrc
            Sets the source reference on Chart Studio Cloud for
            categoryarray .
        categoryorder
            Specifies the ordering logic for the case of
            categorical variables. By default, plotly uses "trace",
            which specifies the order that is present in the data
            supplied. Set `categoryorder` to *category ascending*
            or *category descending* if order should be determined
            by the alphanumerical order of the category names. Set
            `categoryorder` to "array" to derive the ordering from
            the attribute `categoryarray`. If a category is not
            found in the `categoryarray` array, the sorting
            behavior for that attribute will be identical to the
            "trace" mode. The unspecified categories will follow
            the categories in `categoryarray`. Set `categoryorder`
            to *total ascending* or *total descending* if order
            should be determined by the numerical order of the
            values. Similarly, the order can be determined by the
            min, max, sum, mean or median of all the values.
        color
            Sets default for all colors associated with this axis
            all at once: line, font, tick, and grid colors. Grid
            color is lightened by blending this with the plot
            background Individual pieces can override this.
        constrain
            If this axis needs to be compressed (either due to its
            own `scaleanchor` and `scaleratio` or those of the
            other axis), determines how that happens: by increasing
            the "range" (default), or by decreasing the "domain".
        constraintoward
            If this axis needs to be compressed (either due to its
            own `scaleanchor` and `scaleratio` or those of the
            other axis), determines which direction we push the
            originally specified plot area. Options are "left",
            "center" (default), and "right" for x axes, and "top",
            "middle" (default), and "bottom" for y axes.
        dividercolor
            Sets the color of the dividers Only has an effect on
            "multicategory" axes.
        dividerwidth
            Sets the width (in px) of the dividers Only has an
            effect on "multicategory" axes.
        domain
            Sets the domain of this axis (in plot fraction).
        dtick
            Sets the step in-between ticks on this axis. Use with
            `tick0`. Must be a positive number, or special strings
            available to "log" and "date" axes. If the axis `type`
            is "log", then ticks are set every 10^(n*dtick) where n
            is the tick number. For example, to set a tick mark at
            1, 10, 100, 1000, ... set dtick to 1. To set tick marks
            at 1, 100, 10000, ... set dtick to 2. To set tick marks
            at 1, 5, 25, 125, 625, 3125, ... set dtick to
            log_10(5), or 0.69897000433. "log" has several special
            values; "L<f>", where `f` is a positive number, gives
            ticks linearly spaced in value (but not position). For
            example `tick0` = 0.1, `dtick` = "L0.5" will put ticks
            at 0.1, 0.6, 1.1, 1.6 etc. To show powers of 10 plus
            small digits between, use "D1" (all digits) or "D2"
            (only 2 and 5). `tick0` is ignored for "D1" and "D2".
            If the axis `type` is "date", then you must convert the
            time to milliseconds. For example, to set the interval
            between ticks to one day, set `dtick` to 86400000.0.
            "date" also has special values "M<n>" gives ticks
            spaced by a number of months. `n` must be a positive
            integer. To set ticks on the 15th of every third month,
            set `tick0` to "2000-01-15" and `dtick` to "M3". To set
            ticks every 4 years, set `dtick` to "M48"
        exponentformat
            Determines a formatting rule for the tick exponents.
            For example, consider the number 1,000,000,000. If
            "none", it appears as 1,000,000,000. If "e", 1e+9. If
            "E", 1E+9. If "power", 1x10^9 (with 9 in a super
            script). If "SI", 1G. If "B", 1B.
        fixedrange
            Determines whether or not this axis is zoom-able. If
            true, then zoom is disabled.
        gridcolor
            Sets the color of the grid lines.
        gridwidth
            Sets the width (in px) of the grid lines.
        hoverformat
            Sets the hover text formatting rule using d3 formatting
            mini-languages which are very similar to those in
            Python. For numbers, see:
            https://github.com/d3/d3-3.x-api-
            reference/blob/master/Formatting.md#d3_format And for
            dates see: https://github.com/d3/d3-time-
            format#locale_format We add one item to d3's date
            formatter: "%{n}f" for fractional seconds with n
            digits. For example, *2016-10-13 09:15:23.456* with
            tickformat "%H~%M~%S.%2f" would display "09~15~23.46"
        layer
            Sets the layer on which this axis is displayed. If
            *above traces*, this axis is displayed above all the
            subplot's traces If *below traces*, this axis is
            displayed below all the subplot's traces, but above the
            grid lines. Useful when used together with scatter-like
            traces with `cliponaxis` set to False to show markers
            and/or text nodes above this axis.
        linecolor
            Sets the axis line color.
        linewidth
            Sets the width (in px) of the axis line.
        matches
            If set to another axis id (e.g. `x2`, `y`), the range
            of this axis will match the range of the corresponding
            axis in data-coordinates space. Moreover, matching axes
            share auto-range values, category lists and histogram
            auto-bins. Note that setting axes simultaneously in
            both a `scaleanchor` and a `matches` constraint is
            currently forbidden. Moreover, note that matching axes
            must have the same `type`.
        minexponent
            Hide SI prefix for 10^n if |n| is below this number.
            This only has an effect when `tickformat` is "SI" or
            "B".
        mirror
            Determines if the axis lines or/and ticks are mirrored
            to the opposite side of the plotting area. If True, the
            axis lines are mirrored. If "ticks", the axis lines and
            ticks are mirrored. If False, mirroring is disable. If
            "all", axis lines are mirrored on all shared-axes
            subplots. If "allticks", axis lines and ticks are
            mirrored on all shared-axes subplots.
        nticks
            Specifies the maximum number of ticks for the
            particular axis. The actual number of ticks will be
            chosen automatically to be less than or equal to
            `nticks`. Has an effect only if `tickmode` is set to
            "auto".
        overlaying
            If set a same-letter axis id, this axis is overlaid on
            top of the corresponding same-letter axis, with traces
            and axes visible for both axes. If False, this axis
            does not overlay any same-letter axes. In this case,
            for axes with overlapping domains only the highest-
            numbered axis will be visible.
        position
            Sets the position of this axis in the plotting space
            (in normalized coordinates). Only has an effect if
            `anchor` is set to "free".
        range
            Sets the range of this axis. If the axis `type` is
            "log", then you must take the log of your desired range
            (e.g. to set the range from 1 to 100, set the range
            from 0 to 2). If the axis `type` is "date", it should
            be date strings, like date data, though Date objects
            and unix milliseconds will be accepted and converted to
            strings. If the axis `type` is "category", it should be
            numbers, using the scale where each category is
            assigned a serial number from zero in the order it
            appears.
        rangebreaks
            A tuple of
            :class:`plotly.graph_objects.layout.yaxis.Rangebreak`
            instances or dicts with compatible properties
        rangebreakdefaults
            When used in a template (as
            layout.template.layout.yaxis.rangebreakdefaults), sets
            the default property values to use for elements of
            layout.yaxis.rangebreaks
        rangemode
            If "normal", the range is computed in relation to the
            extrema of the input data. If *tozero*`, the range
            extends to 0, regardless of the input data If
            "nonnegative", the range is non-negative, regardless of
            the input data. Applies only to linear axes.
        scaleanchor
            If set to another axis id (e.g. `x2`, `y`), the range
            of this axis changes together with the range of the
            corresponding axis such that the scale of pixels per
            unit is in a constant ratio. Both axes are still
            zoomable, but when you zoom one, the other will zoom
            the same amount, keeping a fixed midpoint. `constrain`
            and `constraintoward` determine how we enforce the
            constraint. You can chain these, ie `yaxis:
            {scaleanchor: *x*}, xaxis2: {scaleanchor: *y*}` but you
            can only link axes of the same `type`. The linked axis
            can have the opposite letter (to constrain the aspect
            ratio) or the same letter (to match scales across
            subplots). Loops (`yaxis: {scaleanchor: *x*}, xaxis:
            {scaleanchor: *y*}` or longer) are redundant and the
            last constraint encountered will be ignored to avoid
            possible inconsistent constraints via `scaleratio`.
            Note that setting axes simultaneously in both a
            `scaleanchor` and a `matches` constraint is currently
            forbidden.
        scaleratio
            If this axis is linked to another by `scaleanchor`,
            this determines the pixel to unit scale ratio. For
            example, if this value is 10, then every unit on this
            axis spans 10 times the number of pixels as a unit on
            the linked axis. Use this for example to create an
            elevation profile where the vertical scale is
            exaggerated a fixed amount with respect to the
            horizontal.
        separatethousands
            If "true", even 4-digit integers are separated
        showdividers
            Determines whether or not a dividers are drawn between
            the category levels of this axis. Only has an effect on
            "multicategory" axes.
        showexponent
            If "all", all exponents are shown besides their
            significands. If "first", only the exponent of the
            first tick is shown. If "last", only the exponent of
            the last tick is shown. If "none", no exponents appear.
        showgrid
            Determines whether or not grid lines are drawn. If
            True, the grid lines are drawn at every tick mark.
        showline
            Determines whether or not a line bounding this axis is
            drawn.
        showspikes
            Determines whether or not spikes (aka droplines) are
            drawn for this axis. Note: This only takes affect when
            hovermode = closest
        showticklabels
            Determines whether or not the tick labels are drawn.
        showtickprefix
            If "all", all tick labels are displayed with a prefix.
            If "first", only the first tick is displayed with a
            prefix. If "last", only the last tick is displayed with
            a suffix. If "none", tick prefixes are hidden.
        showticksuffix
            Same as `showtickprefix` but for tick suffixes.
        side
            Determines whether a x (y) axis is positioned at the
            "bottom" ("left") or "top" ("right") of the plotting
            area.
        spikecolor
            Sets the spike color. If undefined, will use the series
            color
        spikedash
            Sets the dash style of lines. Set to a dash type string
            ("solid", "dot", "dash", "longdash", "dashdot", or
            "longdashdot") or a dash length list in px (eg
            "5px,10px,2px,2px").
        spikemode
            Determines the drawing mode for the spike line If
            "toaxis", the line is drawn from the data point to the
            axis the  series is plotted on. If "across", the line
            is drawn across the entire plot area, and supercedes
            "toaxis". If "marker", then a marker dot is drawn on
            the axis the series is plotted on
        spikesnap
            Determines whether spikelines are stuck to the cursor
            or to the closest datapoints.
        spikethickness
            Sets the width (in px) of the zero line.
        tick0
            Sets the placement of the first tick on this axis. Use
            with `dtick`. If the axis `type` is "log", then you
            must take the log of your starting tick (e.g. to set
            the starting tick to 100, set the `tick0` to 2) except
            when `dtick`=*L<f>* (see `dtick` for more info). If the
            axis `type` is "date", it should be a date string, like
            date data. If the axis `type` is "category", it should
            be a number, using the scale where each category is
            assigned a serial number from zero in the order it
            appears.
        tickangle
            Sets the angle of the tick labels with respect to the
            horizontal. For example, a `tickangle` of -90 draws the
            tick labels vertically.
        tickcolor
            Sets the tick color.
        tickfont
            Sets the tick font.
        tickformat
            Sets the tick label formatting rule using d3 formatting
            mini-languages which are very similar to those in
            Python. For numbers, see:
            https://github.com/d3/d3-3.x-api-
            reference/blob/master/Formatting.md#d3_format And for
            dates see: https://github.com/d3/d3-time-
            format#locale_format We add one item to d3's date
            formatter: "%{n}f" for fractional seconds with n
            digits. For example, *2016-10-13 09:15:23.456* with
            tickformat "%H~%M~%S.%2f" would display "09~15~23.46"
        tickformatstops
            A tuple of :class:`plotly.graph_objects.layout.yaxis.Ti
            ckformatstop` instances or dicts with compatible
            properties
        tickformatstopdefaults
            When used in a template (as
            layout.template.layout.yaxis.tickformatstopdefaults),
            sets the default property values to use for elements of
            layout.yaxis.tickformatstops
        ticklabelmode
            Determines where tick labels are drawn with respect to
            their corresponding ticks and grid lines. Only has an
            effect for axes of `type` "date" When set to "period",
            tick labels are drawn in the middle of the period
            between ticks.
        ticklen
            Sets the tick length (in px).
        tickmode
            Sets the tick mode for this axis. If "auto", the number
            of ticks is set via `nticks`. If "linear", the
            placement of the ticks is determined by a starting
            position `tick0` and a tick step `dtick` ("linear" is
            the default value if `tick0` and `dtick` are provided).
            If "array", the placement of the ticks is set via
            `tickvals` and the tick text is `ticktext`. ("array" is
            the default value if `tickvals` is provided).
        tickprefix
            Sets a tick label prefix.
        ticks
            Determines whether ticks are drawn or not. If "", this
            axis' ticks are not drawn. If "outside" ("inside"),
            this axis' are drawn outside (inside) the axis lines.
        tickson
            Determines where ticks and grid lines are drawn with
            respect to their corresponding tick labels. Only has an
            effect for axes of `type` "category" or
            "multicategory". When set to "boundaries", ticks and
            grid lines are drawn half a category to the left/bottom
            of labels.
        ticksuffix
            Sets a tick label suffix.
        ticktext
            Sets the text displayed at the ticks position via
            `tickvals`. Only has an effect if `tickmode` is set to
            "array". Used with `tickvals`.
        ticktextsrc
            Sets the source reference on Chart Studio Cloud for
            ticktext .
        tickvals
            Sets the values at which ticks on this axis appear.
            Only has an effect if `tickmode` is set to "array".
            Used with `ticktext`.
        tickvalssrc
            Sets the source reference on Chart Studio Cloud for
            tickvals .
        tickwidth
            Sets the tick width (in px).
        title
            :class:`plotly.graph_objects.layout.yaxis.Title`
            instance or dict with compatible properties
        titlefont
            Deprecated: Please use layout.yaxis.title.font instead.
            Sets this axis' title font. Note that the title's font
            used to be customized by the now deprecated `titlefont`
            attribute.
        type
            Sets the axis type. By default, plotly attempts to
            determined the axis type by looking into the data of
            the traces that referenced the axis in question.
        uirevision
            Controls persistence of user-driven changes in axis
            `range`, `autorange`, and `title` if in `editable:
            true` configuration. Defaults to `layout.uirevision`.
        visible
            A single toggle to hide the axis while preserving
            interaction like dragging. Default is true when a
            cheater plot is present on the axis, otherwise false
        zeroline
            Determines whether or not a line is drawn at along the
            0 value of this axis. If True, the zero line is drawn
            on top of the grid lines.
        zerolinecolor
            Sets the line color of the zero line.
        zerolinewidth
            Sets the width (in px) of the zero line.
        